# Solutions to Problem Set 1 in "Dynamics: Theory and Applications, Kane and Levinson 1985"

In [1]:
import sympy as sm
import sympy.physics.mechanics as me

# Problem 1.1

I recommend doing this problem by inspection of the graphic. It will take time but it will help you visualize the contributions to the vectors. Take $\mathbf{u}=\mathbf{a}_1 + 2\mathbf{a}_2+3\mathbf{a}_3$ for example. This vector is fixed in reference frame $A$, so the vector is not a function of any of the variables in $A$. If you then view the vector from $B$ it is apparent that changing $q_1$ can affect the direction of $u$ in $B$. Viewing from $C$ and $D$ will each add $q_2$ and $q_3$ as dependent variables, respectively.

For v it is important to note that $b_1$ and $c_2$ are always orthogonal, i.e. $q_2$ doesn't change them. Same with $d_3$ and $c_2$, orthogononal.

You can use SymPy to check your work though.

Create the three variables for the positive angles.

In [2]:
q1, q2, q3 = sm.symbols('q1:4')

Create the four reference frames by chaining simple rotations through each angle about the axis of rotation.

In [3]:
A = me.ReferenceFrame('A')
B = A.orientnew('B', 'Axis', (q1, A.x))
C = B.orientnew('C', 'Axis', (q2, B.y))
D = C.orientnew('D', 'Axis', (q3, C.z))

D.dcm(A)

In [4]:
D.dcm?

Signature: D.dcm(otherframe)
Docstring:
The direction cosine matrix between frames.

This gives the DCM between this frame and the otherframe.
The format is N.xyz = N.dcm(B) * B.xyz
A SymPy Matrix is returned.

Parameters

otherframe : ReferenceFrame
    The otherframe which the DCM is generated to.

Examples

>>> from sympy.physics.vector import ReferenceFrame, Vector
>>> from sympy import symbols
>>> q1 = symbols('q1')
>>> N = ReferenceFrame('N')
>>> A = N.orientnew('A', 'Axis', [q1, N.x])
>>> N.dcm(A)
Matrix([
[1,       0,        0],
[0, cos(q1), -sin(q1)],
[0, sin(q1),  cos(q1)]])
File:      ~/miniconda3/lib/python3.6/site-packages/sympy/physics/vector/frame.py
Type:      method


Create the three vectors.

In [5]:
u = A.x + 2*A.y + 3*A.z
v = B.x + C.y + D.z
w = D.x + 2*D.y + q3*D.z

In [6]:
u

A.x + 2*A.y + 3*A.z

In [7]:
v

B.x + C.y + D.z

In [8]:
w

D.x + 2*D.y + q3*D.z

You can determine if the variables are present in each vector by expressing each vector in the desired reference frame and inspecting the expression for the variables of interest. For example, for vector $\mathbf{w}$ in reference frame $A$.

In [9]:
w.express(A)

(q3*sin(q2) - 2*sin(q3)*cos(q2) + cos(q2)*cos(q3))*A.x + (-q3*sin(q1)*cos(q2) - 2*sin(q1)*sin(q2)*sin(q3) + sin(q1)*sin(q2)*cos(q3) + sin(q3)*cos(q1) + 2*cos(q1)*cos(q3))*A.y + (q3*cos(q1)*cos(q2) + sin(q1)*sin(q3) + 2*sin(q1)*cos(q3) + 2*sin(q2)*sin(q3)*cos(q1) - sin(q2)*cos(q1)*cos(q3))*A.z

The `free_symbols()` method of the vector can show all unique variables in the expression. You have to supply the reference frame. 

In [10]:
w.free_symbols(A)

{q1, q2, q3}

Try $\mathbf{v}$ in $B$:

In [11]:
v.express(B)

(sin(q2) + 1)*B.x + B.y + cos(q2)*B.z

In [12]:
v.free_symbols(B)

{q2}

Another way to check is to take the derivative of the vector with respect to the variable of interest in the reference frame of interest. If the derivative is zero then the variable is not present. The following nested loop prints `N` or `Y` if the derivative is zero or not. The derivative of the vector must be simplified to compare to zero. Note that simplify is a slow operation and is not guaranteed to simplify completely.

In [13]:
u.diff(q1, A)

0

In [14]:
v.diff(q1, A).simplify()

- sin(q2)*C.x + cos(q2)*C.z - sin(q3)*cos(q2)*D.x - cos(q2)*cos(q3)*D.y

In [15]:
for frame in [A, B, C, D]:
    for variable in [q1, q2, q3]:
        for vector, vec_name in zip([u, v, w], ['u', 'v', 'w']):
            if vector.diff(variable, frame).simplify() != sm.S(0):
                ans = 'Y'
            else:
                ans = 'N'
            print('{}, {}, {}: {}'.format(frame, variable, vec_name, ans))

A, q1, u: N
A, q1, v: Y
A, q1, w: Y
A, q2, u: N
A, q2, v: Y
A, q2, w: Y
A, q3, u: N
A, q3, v: N
A, q3, w: Y
B, q1, u: Y
B, q1, v: N
B, q1, w: N
B, q2, u: N
B, q2, v: Y
B, q2, w: Y
B, q3, u: N
B, q3, v: N
B, q3, w: Y
C, q1, u: Y
C, q1, v: N
C, q1, w: N
C, q2, u: Y
C, q2, v: Y
C, q2, w: N
C, q3, u: N
C, q3, v: N
C, q3, w: Y
D, q1, u: Y
D, q1, v: N
D, q1, w: N
D, q2, u: Y
D, q2, v: Y
D, q2, w: N
D, q3, u: Y
D, q3, v: Y
D, q3, w: Y


# Problem 1.2

Make $q_1$ a function of time and determine which reference frames $\mathbf{v}$ is a function of $t$ in.

In [16]:
q1 = me.dynamicsymbols('q1')

In [17]:
q2, q3 = sm.symbols('q2, q3')

In [18]:
A = me.ReferenceFrame('A')
B = A.orientnew('B', 'Axis', (q1, A.x))
C = B.orientnew('C', 'Axis', (q2, B.y))
D = C.orientnew('D', 'Axis', (q3, C.z))

You can access the default time variable that `dynamicsymbols()` uses like so:

In [19]:
t = me.dynamicsymbols._t

In [20]:
v = B.x + C.y + D.z

In [21]:
v.diff(t, A).simplify()

- sin(q2)*q1'*C.x + cos(q2)*q1'*C.z - sin(q3)*cos(q2)*q1'*D.x - cos(q2)*cos(q3)*q1'*D.y

In [22]:
v.diff(t, B).simplify()

0

In [23]:
v.diff(t, C).simplify()

0

In [24]:
v.diff(t, D).simplify()

0

Only in $A$ does $\mathbf{v}$ have a time derivative, thus $A$ is the answer.

# Problem 1.3

This problem can be solved in the same way as 1.2 using SymPy. You can also think about whether each component of $\mathbf{v}$ will change if $q_2$ changes from each frame. From $A$ $\mathbf{d}_3$ will change. From $B$ $\mathbf{d}_3$ will change. From $C$ $\mathbf{b}_1$ will change. From $D$ $\mathbf{b}_1$ will change, thus it is a function of time in all frames.  

In [25]:
q2 = me.dynamicsymbols('q2')

In [26]:
q1, q3 = sm.symbols('q1, q3')

In [27]:
A = me.ReferenceFrame('A')
B = A.orientnew('B', 'Axis', (q1, A.x))
C = B.orientnew('C', 'Axis', (q2, B.y))
D = C.orientnew('D', 'Axis', (q3, C.z))

In [28]:
t = me.dynamicsymbols._t

In [29]:
v = B.x + C.y + D.z

In [30]:
v.diff(t, A).simplify()

cos(q3)*q2'*D.x - sin(q3)*q2'*D.y

In [31]:
v.diff(t, B).simplify()

cos(q3)*q2'*D.x - sin(q3)*q2'*D.y

In [32]:
v.diff(t, C).simplify()

q2'*B.z

In [33]:
v.diff(t, D).simplify()

q2'*B.z

The derivatives have non-zero values in all frames.

# Problem 1.4

In [34]:
A = me.ReferenceFrame('A')

Create some arbitrary scalars.

In [35]:
a1, a2, a3 = sm.symbols('alpha1:4')
b1, b2, b3 = sm.symbols('beta1:4')
g1, g2, g3 = sm.symbols('gamma1:4')

Create the three vectors using the arbitrary scalars.

In [36]:
alpha = a1 * A.x + a2 * A.y + a3 * A.z
alpha

alpha1*A.x + alpha2*A.y + alpha3*A.z

In [37]:
beta = b1 * A.x + b2 * A.y + b3 * A.z
beta

beta1*A.x + beta2*A.y + beta3*A.z

In [38]:
gamma = g1 * A.x + g2 * A.y + g3 * A.z
gamma

gamma1*A.x + gamma2*A.y + gamma3*A.z

The problem implies that the scalar triple product is commutative and associative. First check if crossing $\alpha$ with $\beta$ before dotting with gamma is equivalent to crossing beta with gamma and dotting with alpha. These can be shown to equivalent by doing the operations and expanding the resulting expressions.

In [39]:
me.dot(alpha, me.cross(beta, gamma)).expand()

alpha1*beta2*gamma3 - alpha1*beta3*gamma2 - alpha2*beta1*gamma3 + alpha2*beta3*gamma1 + alpha3*beta1*gamma2 - alpha3*beta2*gamma1

In [40]:
me.dot(me.cross(alpha, beta), gamma).expand()

alpha1*beta2*gamma3 - alpha1*beta3*gamma2 - alpha2*beta1*gamma3 + alpha2*beta3*gamma1 + alpha3*beta1*gamma2 - alpha3*beta2*gamma1

Similarly the gamma alpha beta and beta gamma alpha roder can also been shown to be equivalent.

In [41]:
me.dot(gamma, me.cross(alpha, beta)).expand()

alpha1*beta2*gamma3 - alpha1*beta3*gamma2 - alpha2*beta1*gamma3 + alpha2*beta3*gamma1 + alpha3*beta1*gamma2 - alpha3*beta2*gamma1

In [42]:
me.dot(beta, me.cross(gamma, alpha)).expand()

alpha1*beta2*gamma3 - alpha1*beta3*gamma2 - alpha2*beta1*gamma3 + alpha2*beta3*gamma1 + alpha3*beta1*gamma2 - alpha3*beta2*gamma1

# Problem 1.5

We've already caculated these derivatives above, so we do it again but take the magnitudes of the vectors. Some need simplification to get the book result.

In [43]:
q1, q2, q3 = sm.symbols('q1:4')
A = me.ReferenceFrame('A')
B = A.orientnew('B', 'Axis', (q1, A.x))
C = B.orientnew('C', 'Axis', (q2, B.y))
D = C.orientnew('D', 'Axis', (q3, C.z))
v = B.x + C.y + D.z

In [44]:
v

B.x + C.y + D.z

In [45]:
v.express(A)

(sin(q2) + 1)*A.x + (-sin(q1)*cos(q2) + cos(q1))*A.y + (sin(q1) + cos(q1)*cos(q2))*A.z

In [46]:
v.diff(q1, A).simplify().express(C)

- sin(q2)*C.x + (-sin(q3)**2*cos(q2) - cos(q2)*cos(q3)**2)*C.y + cos(q2)*C.z

In [47]:
v.diff(q1, A).magnitude().simplify()

sqrt(cos(q2)**2 + 1)

In [48]:
v.magnitude()

sqrt(2*sin(q2) + 3)

In [49]:
type(v.magnitude())

sympy.core.power.Pow

In [50]:
type(q1 + q2)

sympy.core.add.Add

In [51]:
type(q1*q2)

sympy.core.mul.Mul

This should be the same as differentiating the measure numbers!

In [52]:
sm.sqrt(v.dot(A.x).diff(q1)**2 + v.dot(A.y).diff(q1)**2 + v.dot(A.z).diff(q1)**2).simplify()

sqrt(cos(q2)**2 + 1)

In [53]:
v.diff(q1, B).magnitude()

0

In [54]:
v.diff(q2, C).magnitude().simplify()

1

In [55]:
v.diff(q3, C).magnitude()

0

In [56]:
v.diff(q2, D).magnitude().simplify()

1

In [57]:
v.diff(q1, D).magnitude()

0

# Problem 1.6

In [58]:
q1, q2, q3 = sm.symbols('q1:4')
A = me.ReferenceFrame('A')
B = A.orientnew('B', 'Axis', (q1, A.x))
C = B.orientnew('C', 'Axis', (q2, B.y))
D = C.orientnew('D', 'Axis', (q3, C.z))
u = A.x + 2*A.y + 3*A.z

The measure numbers can be found by expressing a vector in the desired reference frame and inspecting the expression for the scalars of each of the three unit vectors.

In [59]:
u.diff(q1, B).express(A).simplify()

3*A.y - 2*A.z

Each measure number can be extracted by using the dot product with each unit vector, i.e. finding the magnitude of the projection of the vector on that axis.

In [60]:
u.diff(q1, B).dot(A.x)

0

In [61]:
u.diff(q1, B).dot(A.y).simplify()

3

In [62]:
u.diff(q1, B).dot(A.z).simplify()

-2

Similiarly for the other two derivatives.

In [63]:
u.diff(q1, B).express(B).simplify()

(-2*sin(q1) + 3*cos(q1))*B.y + (-3*sin(q1) - 2*cos(q1))*B.z

In [64]:
u.diff(q1, A).express(A).simplify()

0

There is convenience method for getting measure numbers in matrix form too:

In [65]:
u.diff(q1, B).to_matrix(A).simplify()

Matrix([
[ 0],
[ 3],
[-2]])

# Problem 1.7

In [66]:
q1, q2, q3 = sm.symbols('q1:4')
A = me.ReferenceFrame('A')
B = A.orientnew('B', 'Axis', (q1, A.x))
C = B.orientnew('C', 'Axis', (q2, B.y))
D = C.orientnew('D', 'Axis', (q3, C.z))

In [67]:
alpha = q1 + q2 + q3
beta = q1**2 + q2**2 + q3**2
gamma = q1**3 + q2**3 + q3**3

In [68]:
r_PQ = alpha * A.x +  beta * B.y + gamma * C.z

In [69]:
r_PQ

(q1 + q2 + q3)*A.x + (q1**2 + q2**2 + q3**2)*B.y + (q1**3 + q2**3 + q3**3)*C.z

In [70]:
dr_dq3_D = r_PQ.diff(q3, D).simplify()
dr_dq3_D

A.x - (q1 + q2 + q3)*cos(q1)*cos(q2)*A.y - (q1 + q2 + q3)*sin(q1)*cos(q2)*A.z + (q1**2 + q2**2 + q3**2)*cos(q2)*B.x + 2*q3*B.y - (q1**2 + q2**2 + q3**2)*sin(q2)*B.z + 3*q3**2*C.z

In [71]:
dr_dq3_D.subs({q1: sm.pi/2, q2: 0, q3: 0}).magnitude()

sqrt(1 + pi**4/16 + 3*pi**2/4)

# Problem 1.8

In [72]:
A = me.ReferenceFrame('A')

In [73]:
v1, v2, v3 = me.dynamicsymbols('v1:4')

In [74]:
v = v1 * A.x + v2 * A.y + v3 * A.z

In [75]:
v.magnitude() * v.magnitude().diff(t)

v1(t)*Derivative(v1(t), t) + v2(t)*Derivative(v2(t), t) + v3(t)*Derivative(v3(t), t)

In [76]:
me.dot(v, v.dt(A))

v1(t)*Derivative(v1(t), t) + v2(t)*Derivative(v2(t), t) + v3(t)*Derivative(v3(t), t)

A unit vector aligned with $\mathbf{v}$ for all time can be found by dividing by the magnitude:

In [77]:
v / v.magnitude()

v1/sqrt(v1**2 + v2**2 + v3**2)*A.x + v2/sqrt(v1**2 + v2**2 + v3**2)*A.y + v3/sqrt(v1**2 + v2**2 + v3**2)*A.z

The `normalize()` method of vectors does this operation:

In [78]:
u = v.normalize()

In [79]:
u

v1/sqrt(v1**2 + v2**2 + v3**2)*A.x + v2/sqrt(v1**2 + v2**2 + v3**2)*A.y + v3/sqrt(v1**2 + v2**2 + v3**2)*A.z

In [80]:
u.dt(A)

((-v1*v1' - v2*v2' - v3*v3')*v1/(v1**2 + v2**2 + v3**2)**(3/2) + v1'/sqrt(v1**2 + v2**2 + v3**2))*A.x + ((-v1*v1' - v2*v2' - v3*v3')*v2/(v1**2 + v2**2 + v3**2)**(3/2) + v2'/sqrt(v1**2 + v2**2 + v3**2))*A.y + ((-v1*v1' - v2*v2' - v3*v3')*v3/(v1**2 + v2**2 + v3**2)**(3/2) + v3'/sqrt(v1**2 + v2**2 + v3**2))*A.z

To show that the expression in the book is the same:

In [81]:
v.dt(A) / v.magnitude() - v.outer(v).dot(v.dt(A))/v.magnitude()**3

(-(v1**2*v1' + v1*v2*v2' + v1*v3*v3')/(v1**2 + v2**2 + v3**2)**(3/2) + v1'/sqrt(v1**2 + v2**2 + v3**2))*A.x + (-(v1*v2*v1' + v2**2*v2' + v2*v3*v3')/(v1**2 + v2**2 + v3**2)**(3/2) + v2'/sqrt(v1**2 + v2**2 + v3**2))*A.y + (-(v1*v3*v1' + v2*v3*v2' + v3**2*v3')/(v1**2 + v2**2 + v3**2)**(3/2) + v3'/sqrt(v1**2 + v2**2 + v3**2))*A.z

# Problem 1.9

In [82]:
q1, q2 = sm.symbols('q1, q2')

In [83]:
f = sm.Function('f')(q1, q2)
f

f(q1, q2)

In [84]:
g = sm.Function('g')(q1, q2)
g

g(q1, q2)

In [85]:
h = sm.Function('h')
type(h)
h(q1, q2).diff(q1)

Derivative(h(q1, q2), q1)

In [86]:
theta = sm.Function('theta')(q1, q2)

In [87]:
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')

In [88]:
B.orient(A, 'Axis', (theta, A.z))

In [89]:
v = f * A.x + g * A.y
v

f(q1, q2)*A.x + g(q1, q2)*A.y

Left side of the equation simplifies to the expected expression:

In [90]:
(v.diff(q2, B).diff(q1, A) - v.diff(q1, A).diff(q2, B)).simplify()

g(q1, q2)*Derivative(theta(q1, q2), q1, q2)*A.x - f(q1, q2)*Derivative(theta(q1, q2), q1, q2)*A.y

Make sure you can get these result when taking the derivatives (it is worth doing some by hand):

In [91]:
v.diff(q2, B).simplify()

(g(q1, q2)*Derivative(theta(q1, q2), q2) + Derivative(f(q1, q2), q2))*A.x + (-f(q1, q2)*Derivative(theta(q1, q2), q2) + Derivative(g(q1, q2), q2))*A.y

In [92]:
v.diff(q2, B).diff(q1, A).simplify()

(g(q1, q2)*Derivative(theta(q1, q2), q1, q2) + Derivative(g(q1, q2), q1)*Derivative(theta(q1, q2), q2) + Derivative(f(q1, q2), q1, q2))*A.x + (-f(q1, q2)*Derivative(theta(q1, q2), q1, q2) - Derivative(f(q1, q2), q1)*Derivative(theta(q1, q2), q2) + Derivative(g(q1, q2), q1, q2))*A.y

# Problem 1.10

In [93]:
L = me.ReferenceFrame('L')  # The laboratory frame
C = me.ReferenceFrame('C')  # The disc frame
R = me.ReferenceFrame('R')  # The rod frame

We need the q's to be functions of time so you can use `dynamicsymbols()` to create these more easily that using `Function()` explicitly.

In [94]:
q1, q2 = me.dynamicsymbols('q1, q2')
q1, q2

(q1(t), q2(t))

In [95]:
r = sm.symbols('r', real=True)

I'll assume that when $q_1=0$ the $L$ and $C$ unit vectors are equivalent, so a simple rotation about the 1 axis will orient $C$ with respect to $L$.

In [96]:
C.orient(L, 'Axis', (q1, L.x))

Similarly, I'll assume that when $q_2=0$ the rod points in the $-\mathbf{c}_1$ direction.

In [97]:
R.orient(C, 'Axis', (q2, C.y))

Verify the first two derivatives.

In [98]:
C.y.diff(q1, L).simplify()

C.z

In [99]:
C.z.diff(q1, L).simplify()

- C.y

Form the vector from $O$ to $P$, making use of the two reference frames.

In [100]:
p = r*C.y - 3*r*R.x

In [101]:
p

r*C.y - 3*r*R.x

In [102]:
p.express(C)

- 3*r*cos(q2)*C.x + r*C.y + 3*r*sin(q2)*C.z

In [103]:
p.dt(L)

r*q1'*C.z - 3*r*sin(q2)*q1'*R.y + 3*r*q2'*R.z

The following expression matches the result in the book:

In [104]:
p.dt(L).express(C)

3*r*sin(q2)*q2'*C.x - 3*r*sin(q2)*q1'*C.y + (3*r*cos(q2)*q2' + r*q1')*C.z

# Problem 1.11

Start by creating a vector that has measure numbers that are functions of time.

In [105]:
N = me.ReferenceFrame('N')

In [106]:
a1, a2, a3 = me.dynamicsymbols('a1:4')

In [107]:
v = a1*N.x + a2*N.y + a3*N.z
v

a1*N.x + a2*N.y + a3*N.z

Now create a unit vector that is always aligned with $\mathbf{v}$.

In [108]:
u = v.normalize()
u

a1/sqrt(a1**2 + a2**2 + a3**2)*N.x + a2/sqrt(a1**2 + a2**2 + a3**2)*N.y + a3/sqrt(a1**2 + a2**2 + a3**2)*N.z

We are told that at $t^*$ vector $\mathbf{v}$ has specific properities. We can then infer that:

- $\mathbf{a}_1(t^*)=1,\mathbf{a}_2(t^*)=0,\mathbf{a}_3(t^*)=0$
- $\dot{\mathbf{a}}_1(t^*)=0,\dot{\mathbf{a}}_2(t^*)=1,\dot{\mathbf{a}}_3(t^*)=0$
- $\ddot{\mathbf{a}}_1(t^*)=0,\ddot{\mathbf{a}}_2(t^*)=0,\ddot{\mathbf{a}}_3(t^*)=1$

So twice differentiate $\mathbf{u}$ to get the general expression.

In [109]:
d2udt2 = u.dt(N).dt(N)
d2udt2

((-3*a1*a1' - 3*a2*a2' - 3*a3*a3')*(-a1*a1' - a2*a2' - a3*a3')*a1/(a1**2 + a2**2 + a3**2)**(5/2) + 2*(-a1*a1' - a2*a2' - a3*a3')*a1'/(a1**2 + a2**2 + a3**2)**(3/2) + a1''/sqrt(a1**2 + a2**2 + a3**2) + (-a1*a1'' - a2*a2'' - a3*a3'' - a1'**2 - a2'**2 - a3'**2)*a1/(a1**2 + a2**2 + a3**2)**(3/2))*N.x + ((-3*a1*a1' - 3*a2*a2' - 3*a3*a3')*(-a1*a1' - a2*a2' - a3*a3')*a2/(a1**2 + a2**2 + a3**2)**(5/2) + 2*(-a1*a1' - a2*a2' - a3*a3')*a2'/(a1**2 + a2**2 + a3**2)**(3/2) + a2''/sqrt(a1**2 + a2**2 + a3**2) + (-a1*a1'' - a2*a2'' - a3*a3'' - a1'**2 - a2'**2 - a3'**2)*a2/(a1**2 + a2**2 + a3**2)**(3/2))*N.y + ((-3*a1*a1' - 3*a2*a2' - 3*a3*a3')*(-a1*a1' - a2*a2' - a3*a3')*a3/(a1**2 + a2**2 + a3**2)**(5/2) + 2*(-a1*a1' - a2*a2' - a3*a3')*a3'/(a1**2 + a2**2 + a3**2)**(3/2) + a3''/sqrt(a1**2 + a2**2 + a3**2) + (-a1*a1'' - a2*a2'' - a3*a3'' - a1'**2 - a2'**2 - a3'**2)*a3/(a1**2 + a2**2 + a3**2)**(3/2))*N.z

Now subsitute the values at time $t^*$ and compute the magnitude. Be careful that you substitue the highest derivatives first.

In [110]:
temp1 = d2udt2.subs({a1.diff(t).diff(t): 0, a2.diff(t).diff(t): 0, a3.diff(t).diff(t): 1})
temp2 = temp1.subs({a1.diff(t): 0, a2.diff(t): 1, a3.diff(t): 0})
temp2.subs({a1: 1, a2:0, a3: 0}).magnitude()

sqrt(2)